1. Transformerの紹介

    1.1 なぜTransformerが必要なのか

    RNN：長距離の依存関係情報をキャプチャできますが、並列処理ができません；
    
    CNN：並列処理は可能ですが、長距離の依存関係情報を捉えることが難しい（層を重ねるか、畳み込みkernelを拡張して受容野を大きくする必要があります）；
    
    伝統的なAttention：ソースとターゲットの隠れベクトルに基づいてAttention scoreを計算しますが、主にソースの各単語がターゲットの生成にどのように影響を与えるかに焦点を当てています。ターゲットの単語が生成される際に、それらが相互にどのように影響を及ぼすか（例えば、語順の正確さや文法的正確さを保つため）を充分に捉えることができないため、文の自然さや流暢さが損なわれることがあります。

    この問題を解決するために、Transformerのような新しいモデルでは,multi-head Attentionを用いて、ソースとターゲットの両方の内部依存関係をより詳細に考慮しています。

    1.2 Transformerの特徴

    ニューラルネットワーク内での再帰の使用を避け、完全にself-attentionメカニズムに依存して、入力と出力の間の全体的な依存関係を把握できるものです。

2. Tranformerの構造

    ２.1　全体的な構造

    Transformerのエンコーダーは、複数のエンコーダーレイヤーが積み重ねられて構成されています。デコーダーもエンコーダーと同様に、複数のデコーダーレイヤーから成り立っています。

    ![arch](../../../image/3_NLP/transformer/2-arch-all.png)

    2.2 Encoder

    2.2.1 Encoderの構造

    ![encoder-interal-structure](../../../image/3_NLP/transformer/2-1-transformer_resideual_layer_norm.png)
    
    i. セルフアテンション層（Self-Attention Layer）:
        
    この層は、入力sequence内の他の部分を見ることができるようにして、特定の部分をより良くエンコードすることができます。シーケンス内の各要素が他のすべての要素に対してどれだけ注目すべきかを計算することで機能します。これらのスコアは、「スケーリング済みドット積アテンション」というメカニズムを通じて計算されます。

    計算中に、二つの単語ベクトル間にはRNNのような時間的な前後関係がないため、計算時間がRNNよりも速いです
    
    ii. 前方向フィード全結合層（Feed-Forward Neural Network）:
    
    セルフアテンション層からの出力は、全結合の前方向フィードネットワークに入力されます。このネットワークはシーケンス内の他の位置とは独立しており、全ての位置で同じです（重み共有）。この層は通常、2つの線形変換と1つの活性化関数（例えばReLU）を含みます。

    2.2.2 self-Attentionの計算

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-calculation.jpeg)

    論文ではembedding_length=512, dk=dv=64

    Self-attentionの具体的なステップです：

    i. **embedding**：
    最初に、入力シーケンスの各要素（例えば、文中の各単語）がベクトルに変換されます。これらのベクトルは通常、埋め込み層（例えば、単語埋め込み）を通じて得られます。

    ii. **Q（クエリ）、K（キー）、V（バリュー）ベクトルの生成**：
    入力シーケンスの各要素のベクトルに対して、異なる線形変換を適用して、クエリベクトル、キーベクトル、バリューベクトルの三つの新しいベクトル群を生成します。これらの変換は学習可能な重み行列によって実現されます。

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-calculation-1.jpeg)

    iii. **注意力スコアの計算**：
    各クエリベクトルとすべてのキーベクトルとの間でドット積を計算し、注意力スコアを得ます。これらのスコアは、出力を生成する際に各要素がシーケンス内の他の要素をどれだけ考慮すべきかを決定します。
    
    iv. **スケール済みドット積**：
    トレーニングをより安定させるために、ドット積のスコアをキーベクトルの次元の平方根で割ります。この操作は、スコアが大きすぎることによる勾配消失問題を避けるのに役立ちます。

    v. Softmaxの適用：
    スケール済みのドット積スコアにSoftmax関数を適用し、スコアを確率分布に変換します。これにより、各要素が他の要素をどれだけ重視しているかが表されます。
    
    vi. 重み付け和：
    Softmax出力の注意力重みを対応するバリューベクトルに掛け合わせ、すべての積を合計して、その位置の最終出力ベクトルを生成します。このステップは、注意力重みに基づいてシーケンス全体から情報を集約することに相当します。

    vii. 出力表現：
    各入力要素は新しいベクトル表現を得ます。これらの新しいベクトルは、入力シーケンス全体の情報を含んでおり、次の深層ニューラルネットワーク処理に使用されます。

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-calculation-2.jpeg)

    2.4 Multi-head attention

    多頭注意力メカニズムは、self-attentionを拡張したもので、異なる表現のサブスペースで情報を並行して学習できるようにします。具体的な手順は以下の通りです：

    i. パラメータの分割：
    通常のself-attentionでは、Wq, Wk, Wvという３つの重み行列を使用して、クエリ、キー、バリューのベクトルを生成します。multi-head attentionでは、これらの行列を複数の小さな行列に分割し、各「頭」がそれぞれの行列を使用します。

    ii. 並列self-attentionの計算：
    各頭は独立して自己注意力の計算を行います。つまり、各頭はそれぞれのWq, Wk, Wvを使用して、クエリ、キー、バリューを生成し、注意力スコアを計算します。

    iii. 出力の結合：
    各頭の出力は結合され、追加の線形変換Woを通じて情報が統合されます。これにより、最終的な出力表現が生成されます。

    例：   

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-calculation-3.png)

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-calculation-4.png)

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-overall.jpeg)

    2.2.3 Position encoding

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-position-encoding.jpeg)

    Position encodingがinput sequence内の単語の位置順序を考慮できるようにするための技術です。TransformerモデルはRNNのようにシーケンスの時間ステップ情報を自然に処理するわけではないため、位置情報を提供するために位置エンコーディングが必要です。

    ![self-attention-calculation](../../../image/3_NLP/transformer/2-self-attention-position-encoding-2.png)

    sinとcos関数を利用した理由：
    
    独立性の確保：
    正弦（sin）関数と余弦（cos）関数は互いに直交しているため、独立した信号を生成することができます。偶数次元でsin関数を、奇数次元でcos関数を使用することにより、モデルに豊かな位置情報を提供し、各次元の符号が重複しないようにします。

    相位差の提供：
    sinとcosは90度の相位差を持っています（つまり、周期の四分の一）。この相位差により、連続する次元においても符号が異なり、位置符号の各次元間でシステマティックな違いが生まれます。これにより、モデルは位置の変化に対して敏感になります。

    位置情報の表現能力の向上：
    sinとcosを交互に使用することで、位置の絶対情報だけでなく、位置間の相対的な違いに関する情報も提供されます。例えば、ある次元で二つの位置の符号が非常に近い場合、モデルはこれらの位置がその次元において相対的に近いことを推測できます。

    モデルの汎化能力の向上：
    sinとcosを交互に使用することで、モデルは訓練データにない新しい位置に対しても、一貫性と連続性を持って位置関係を理解し推測することが可能になります。

    2.2.4 details

    正規化層（Normalization Layer）:セルフアテンション層と前方向フィード全結合層の後には通常、正規化層が配置されます。この層は訓練プロセスの安定性を助け、通常は層正規化が用いられます。正規化は、異なる層からの出力のスケールの違いを処理し、訓練速度とモデルの安定性を向上させます。

    残差接続（Residual Connection）:セルフアテンション層と前方向フィード全結合層は、それぞれ残差接続があります。残差接続は、深いネットワークでの勾配の消失問題を緩和するのに役立ち、勾配がネットワークの初期層を直接通過できるようにします。各サブレイヤーの出力は、そのサブレイヤーの入力と出力の和です。

    ![self-attention-calculation](../../../image/3_NLP/transformer/2_self-attention-details.jpeg)

    2.3 Decoder

    2.3.1 decoder内部構造

    ![decoder](../../../image/3_NLP/transformer/3-decoder.jpeg)

    i. **Masked Self-Attention Layer**
    この層では、デコーダーが以前に生成した単語に対してのみ注意を払います。ここでは「マスク」が使用されており、位置 i の出力が位置 i 未満の入力にのみ依存するようにします。これにより、将来の情報を予測時に使用することが防がれます。

    ii. **Encoder-Decoder Attention Layer**
    この層はデコーダーが入力シーケンスのどの部分に注目すべきかを決定するのに役立ちます。ここでのQueryはデコーダーの前層から、KeyとValueはエンコーダーの出力から来ます。これにより、デコーダーは翻訳や文章生成の際に入力のどの部分に注目すべきかを動的に決定できます。

    iii. **Feed-Forward Network**:
    各デコーダー層には、エンコーダー層と同様の前方向フィードネットワークが含まれています。このネットワークは、アテンション層からの出力をさらに処理します。

    2.3.2 計算 
    self-attention層とFeed-Forward Neural Network層がencoderと同じなので、主にencoder-decoder層の計算を紹介します。

    2.3.2.1 パラメーター：
    
    i. **encoderの出力**：
    
    エンコーダーの最終出力 $H$は、入力シーケンスの長さを $N$、モデルの次元を $d_{model}$とすると、$H$の形状は$N \times d_{model}$になります。

    ii. **キーKとバリューV**：

    $K=HW^K$  &ensp; $K \in \mathbb{R}^{N \times d_k}$ &ensp; $W^K \in \mathbb{R}^{d_{model} \times d_k}$ 

    $V=HW^V$ &ensp; $V \in \mathbb{R}^{N \times d_v}$ &ensp; $W^V \in \mathbb{R}^{d_{model} \times d_v}$

    iii. Decoderで作成した $Q$  

    2.3.2.2 Vを計算

    デコーダーは$Q$
  と $K$ のドット積を計算して、各入力要素が現在の出力要素の生成にどれだけ重要かを判断し、その情報を使って $V$ を重み付けし、その層の出力を生成します。



3. 追加内容

![Alt text](image.png)

3.1 Evolutionary tree

![Alt text](image-1.png)


 







